# Week 7 – 模型公平性分析项目

本项目演示了如何使用 **Fairlearn** 库对模型进行公平性评估与改进。我们以美国司法系统数据（ProPublica 数据集）为例，预测个体在两年内是否会再犯，并探讨模型在不同群体（非裔 vs. 非非裔）之间的表现差异。

## 项目背景与目标
在某些应用场景（如司法、医疗、招聘），模型预测可能对不同群体带来不公平影响。我们需要识别和量化这种不公平性，并尝试改进模型，使其在保持良好性能的同时尽量减少群体差异。

## 本项目内容
1. **数据加载与预处理**：读取 ProPublica 提供的数据集，并选取目标列（是否再犯）和敏感属性（非裔与否）。
2. **训练多个模型**：
   - Baseline：普通逻辑回归
   - Postprocessing：ThresholdOptimizer 后处理模型
   - ExponentiatedGradient：在训练过程中直接约束 EqualizedOdds
3. **公平性指标与可视化**：计算 `Equalized Odds`、`FPR`、`FNR` 等指标，并用 Plotly 生成一些图表。
4. **反思**：对过程中使用的工具、遇到的挑战和结果进行总结。

## 主要依赖包
- `pandas`、`numpy` 等用于数据处理
- `scikit-learn` 用于模型训练与评估
- `fairlearn` 用于公平性指标和后处理
- `plotly` 用于可视化

下面我们开始具体的代码示例。

## 1. 导入库与数据加载

**思路**：
1. 我们先导入常用的 Python 数据科学库和公平性库。
2. 从 CSV 文件中读取数据，并查看基本信息。
3. 指定目标列 `y` 和特征 `X`，以及敏感属性 `African_American`。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

from fairlearn.metrics import MetricFrame, false_positive_rate, false_negative_rate, equalized_odds_difference
from fairlearn.postprocessing import ThresholdOptimizer
from fairlearn.reductions import ExponentiatedGradient, EqualizedOdds

import plotly.express as px
import plotly.graph_objects as go

# ============== 数据加载 ==============
data_path = "./data/propublica_data_for_fairml.csv"  # 你需要改成实际文件路径
data = pd.read_csv(data_path)

print("数据列名:", data.columns)
print(data.head())

# ============== 目标变量与特征 ==============
y = data["Two_yr_Recidivism"]
X = data.drop(columns=["Two_yr_Recidivism"])

# 这里我们假设敏感列名叫 'African_American'，你要根据实际列名替换
sensitive_name = "African_American"  # 如果不一样，请修改
sensitive_features = X[sensitive_name]

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

## 2. 训练三个模型

**思路**：
1. **基线模型**：普通逻辑回归，不做任何公平性约束。
2. **后处理模型**：训练完基线后，用 `ThresholdOptimizer` 对输出概率做后处理，强制满足 `equalized_odds`。
3. **ExponentiatedGradient**：在训练阶段直接对 `LogisticRegression` 加入公平性约束。

最后，我们会计算各模型的准确率（Accuracy）、AUC、`Equalized Odds` 指标。

In [ ]:
# ============== (1) 基线模型：逻辑回归 ==============
baseline_model = LogisticRegression(max_iter=1000)
baseline_model.fit(X_train, y_train)

y_pred_baseline = baseline_model.predict(X_test)
y_pred_proba_baseline = baseline_model.predict_proba(X_test)[:, 1]

acc_baseline = accuracy_score(y_test, y_pred_baseline)
auc_baseline = roc_auc_score(y_test, y_pred_proba_baseline)
eod_baseline = equalized_odds_difference(
    y_test, y_pred_baseline,
    sensitive_features=X_test[sensitive_name]
)

# 计算分组指标
metrics = {
    "accuracy": accuracy_score,
    "fpr": false_positive_rate,
    "fnr": false_negative_rate
}
metric_frame_baseline = MetricFrame(
    metrics=metrics,
    y_true=y_test,
    y_pred=y_pred_baseline,
    sensitive_features=X_test[sensitive_name]
)

# ============== (2) 后处理模型：ThresholdOptimizer ==============
post_model = ThresholdOptimizer(
    estimator=baseline_model,
    constraints="equalized_odds",
    objective="accuracy_score",
    prefit=True,
    predict_method="predict_proba"
)
post_model.fit(X_train, y_train, sensitive_features=X_train[sensitive_name])
y_pred_post = post_model.predict(X_test, sensitive_features=X_test[sensitive_name])

# 试图获取概率
try:
    y_pred_proba_post = post_model._pmf_predict(X_test, sensitive_features=X_test[sensitive_name])
    if y_pred_proba_post.ndim == 2:
        y_pred_proba_post = y_pred_proba_post[:, 1]
except:
    y_pred_proba_post = np.zeros_like(y_pred_baseline)

acc_post = accuracy_score(y_test, y_pred_post)
auc_post = (roc_auc_score(y_test, y_pred_proba_post)
            if np.any(y_pred_proba_post) else np.nan)
eod_post = equalized_odds_difference(
    y_test, y_pred_post,
    sensitive_features=X_test[sensitive_name]
)
metric_frame_post = MetricFrame(
    metrics=metrics,
    y_true=y_test,
    y_pred=y_pred_post,
    sensitive_features=X_test[sensitive_name]
)

# ============== (3) 训练时公平约束模型：ExponentiatedGradient ==============
exp_model = ExponentiatedGradient(
    estimator=LogisticRegression(max_iter=1000),
    constraints=EqualizedOdds()
)
exp_model.fit(X_train, y_train, sensitive_features=X_train[sensitive_name])
y_pred_exp = exp_model.predict(X_test)

# 由于 predict_proba 可能不可用，这里只做类别评估
acc_exp = accuracy_score(y_test, y_pred_exp)
auc_exp = roc_auc_score(y_test, y_pred_exp)  # 仅作示例，不是最优做法
eod_exp = equalized_odds_difference(
    y_test, y_pred_exp,
    sensitive_features=X_test[sensitive_name]
)
metric_frame_exp = MetricFrame(
    metrics=metrics,
    y_true=y_test,
    y_pred=y_pred_exp,
    sensitive_features=X_test[sensitive_name]
)

# ============== 汇总结果 ==============
results_df = pd.DataFrame({
    "Model": ["Baseline", "Postprocessing", "ExpGradient"],
    "Accuracy": [acc_baseline, acc_post, acc_exp],
    "AUC": [auc_baseline, auc_post, auc_exp],
    "Equalized_Odds": [eod_baseline, eod_post, eod_exp]
})

results_df["Error_Rate"] = 1 - results_df["Accuracy"]
print("\n=== 各模型指标对比 ===")
print(results_df)


## 3. 可视化：Plotly 图表

我们展示几类图：
1. **分布图**：`Two_yr_Recidivism` 在不同群体中的分布
2. **柱状图**：基线模型在不同群体下的准确率、FPR、FNR
3. **ROC 曲线**：比较基线模型和后处理模型
4. **散点图**：模型的性能（错误率）与公平性（Equalized Odds）权衡

在 Jupyter Notebook 中，Plotly 可以用 `fig.show()` 直接在单元格中交互显示。

In [ ]:
# ============== (A) 分布图 ==============
fig_dist = px.histogram(
    data, x="Two_yr_Recidivism", color=sensitive_name, barmode="overlay",
    histnorm='density', opacity=0.6,
    title="两年内再犯分布（按非裔 vs 非非裔）"
)
fig_dist.update_layout(xaxis_title="Two_yr_Recidivism", yaxis_title="Density")
fig_dist.show()

# ============== (B) 柱状图：基线模型的分组指标 ==============
group_metrics = metric_frame_baseline.by_group.reset_index()
fig_bar = go.Figure()
for metric in ["accuracy", "fpr", "fnr"]:
    fig_bar.add_trace(go.Bar(
        x=group_metrics[sensitive_name].astype(str),
        y=group_metrics[metric],
        name=metric
    ))
fig_bar.update_layout(
    barmode="group",
    title="基线模型：不同群体的性能指标",
    xaxis_title=f"{sensitive_name} (0=非, 1=是)",
    yaxis_title="指标值"
)
fig_bar.show()

# ============== (C) ROC 曲线：基线 vs 后处理 ==============
fpr_baseline, tpr_baseline, _ = roc_curve(y_test, y_pred_proba_baseline)

# 如果后处理模型有概率
if np.any(y_pred_proba_post):
    fpr_post, tpr_post, _ = roc_curve(y_test, y_pred_proba_post)
else:
    fpr_post, tpr_post = [], []

fig_roc = go.Figure()
fig_roc.add_trace(go.Scatter(
    x=fpr_baseline, y=tpr_baseline, mode='lines',
    name=f"Baseline (AUC={auc_baseline:.2f})"
))
if len(fpr_post) > 0:
    fig_roc.add_trace(go.Scatter(
        x=fpr_post, y=tpr_post, mode='lines',
        name=f"Postprocessing (AUC={auc_post:.2f})"
    ))
fig_roc.update_layout(
    title="ROC 曲线对比",
    xaxis_title="False Positive Rate",
    yaxis_title="True Positive Rate"
)
fig_roc.show()

# ============== (D) 性能-公平性散点图 ==============
fig_scatter = px.scatter(
    results_df, x="Error_Rate", y="Equalized_Odds", text="Model",
    title="模型性能与公平性权衡图",
    labels={"Error_Rate": "错误率 (1 - Accuracy)", "Equalized_Odds": "Equalized Odds 差异"}
)
fig_scatter.update_traces(textposition='top center')
fig_scatter.show()


## 4. 反思与总结

1. **工具或技术的选择**：
   - 我们使用了 `fairlearn` 库来衡量并改进模型的公平性。这对于涉及不同敏感群体的预测非常重要。
   - 使用 `ThresholdOptimizer`（后处理方法）可以在不重新训练基线模型的情况下进行公平性调优；而 `ExponentiatedGradient` 直接在训练阶段施加约束。

2. **遇到的挑战与解决**：
   - **挑战**：获取概率预测时，一些模型（如 `ExponentiatedGradient`）可能不提供 `predict_proba`。
   - **解决**：暂时用类别预测近似计算 AUC，或者查看官方文档，看看是否能在回调函数中获取 logits。

3. **对数据和统计的理解变化**：
   - 在传统的准确率、AUC 等指标之外，还需关注不同群体的 `FPR`、`FNR` 等，以衡量模型在不同人群中的表现差异。

4. **实践应用**：
   - 在司法、医疗、招聘等高风险领域，对不同敏感群体进行公平性评估至关重要。
   - 若发现模型对某群体具有较高的误报或误拒，就需要进一步分析原因，并做相应的算法或数据层面改进。

5. **伦理影响**：
   - 数据中的偏见可能导致模型加剧社会不公；需要在采集、标注和使用时保持审慎。
   - 在做公平性约束时，也要平衡模型性能与实际应用需求，避免因过度追求公平而忽视整体准确性。

## 最终小结
通过本项目，我们学会了如何：
- 使用 `fairlearn` 评估并改进模型的公平性；
- 对比后处理和训练时约束的两种方法；
- 在可视化中观察不同群体指标的差异，并思考这些差异的成因和影响。

如果你想继续扩展，可以尝试：
- 使用更多敏感属性（如性别、年龄段）做多维分析；
- 尝试更复杂的模型（随机森林、XGBoost 等）并结合公平性方法；
- 深入研究如何在实际应用中平衡公平性和性能。